# GOandUISP
Script Python per la conversione di file _.xlsx_ da output GoAndSwim a input per il portale online UISP.

## Guida rapida (online version)

### Preparare il file di input
1. Creare un file excel (_.xlsx_), e aprirlo con Excel. Questo file deve essere vuoto per il momento
2. Una volta aperto il file, aprire il file _.GAS_ della manifestazione che si vuole convertire
3. Selezionare le gare di interesse ed entrare in esse (tasto in basso a sinistra di GoAndSwim)
4. Senza dover fare nulla, si dovrebbe essere sul primo atleta dell'elenco. A questo punto eseguire la combinazione di tasti _CTRL + C_ (copia)
5. Andare nel file vuoto appena creato nella prima cella in alto a sinistra (la A1) ed eseguirle la combinazione di tasti _CTRL + V_ (incolla). In questo modo tutte le gare di interesse dovrebbero essere state copiate nel file Excel
6. Salvare il file e chiuderlo

### Esecuzione

Premere sul pulsante a forma di cartella sulla sinistra di questo documento e caricare i file che si desidera convertire.
Successivamente, eseguire il codice sotto questo testo con il tasto *play* oppure entrandovi e premendo CTRL + INVIO.
__NOTA__: nel caso di atleti con più di un nome/cognome il programma chiederà di inserirli manualmente *in fondo a questa pagina web*, comunicandovi tutti i dati che possiede di quel determinato atleta. Potete scrivere sia in maiuscolo che in minuscolo ed inserire spazi (per i nomi/cognomi multipli). Una volta inserito il nome/cognome multiplo richiesto, premere INVIO e ripetere l'azione ogni qualvolta richiesta dal programma.

### Output

Una volta eseguito lo script i file saranno formattati correttamente e potranno essere scaricati.
In caso le modifiche non siano visibili, ricaricare la pagina (tasto F5).

In [ ]:
import pandas as pd
import os
__version__ = "1.3.5"
# races dictionary: GoAndSwim -> dbMeeting
__styles__ = {'F': 'Delfino', 'D': 'Dorso', 'R': 'Rana', 'S': 'SL'}
__in_columns__ = ['Name', 'Year', 'Sex', '', 'Distance', 'Style', 'Team'] + \
    [''] * 3 + ['Time'] + [''] * 2 + ['Boolean', 'Absent'] + [''] * 5


def convert(file_name):
    input_file = pd.read_excel(file_name, header=None)
    # drop nomiStaffette column, if exists
    if len(input_file.columns) == 21:
        input_file.drop(input_file.columns[1], axis=1, inplace=True)

    input_file.columns = __in_columns__

    # now print how many athletes are in each team and the total (partecipating medals)
    counter_df = input_file.drop(
        input_file.loc[input_file['Absent'].str.strip() == 'A'].index, inplace=False)
    counter_df = counter_df.groupby(['Name', 'Year', 'Sex', 'Team'])[
        ['Time']].agg(list)

    print(counter_df.index.get_level_values('Team').value_counts())
    print("TOTALE ATLETI PARTECIPANTI: " + str(len(counter_df.index)))

    input("Premere un tasto qualsiasi per continuare...")

    # keep only rows with boolean set to T (valid times) and strip spaces in style column
    input_file.drop(
        input_file.loc[input_file['Boolean'].str.strip() != 'T'].index, inplace=True)
    input_file['Style'] = input_file['Style'].str.strip()
    # keeping only interesting data
    input_file = input_file[['Name', 'Year', 'Sex',
                             'Style', 'Distance', 'Time', 'Team']]
    # replacing style names
    input_file = input_file.replace({'Style': __styles__})
    input_file['Race'] = input_file['Distance'].astype(
        str) + " " + input_file['Style']
    # groupby races and times, i.e. get unique athletes
    input_file = input_file.groupby(['Name', 'Year', 'Sex', 'Team'])[
        ['Race', 'Time']].agg(list)

    # creates empty output database with columns' names
    out_columns = ['Cognome', 'Nome', 'Anno', 'Sesso']
    max_len = input_file['Race'].apply(len).max()
    for i in range(max_len):
        out_columns.append('Gara' + str(i+1))
        out_columns.append('Tempo' + str(i+1))
    out_columns.append('Societa')
    output_file = pd.DataFrame(columns=out_columns)

    output_file['Anno'] = input_file.index.get_level_values('Year')
    output_file['Sesso'] = input_file.index.get_level_values('Sex')
    output_file['Societa'] = input_file.index.get_level_values('Team')

    # get unique athletes
    athletes = input_file.index

    # split name column into words and ask surname in input if the number of words is greater than 2
    for index, full_name in enumerate(athletes.get_level_values('Name')):
        if len(full_name.split()) > 2:
            print("Inserisci i dati di " + str(athletes[index]) + ": ")
            while True:
                surname = input("Inserisci il COGNOME: ").upper()
                if surname in full_name.upper():
                    name = full_name.upper().replace(surname + ' ', '')
                    break
                else:
                    print("COGNOME non presente nel nome, riprova: ")
            output_file.loc[index, 'Nome'] = name
            output_file.loc[index, 'Cognome'] = surname
        else:
            name_column = full_name.split()
            output_file.loc[index, 'Nome'] = name_column[1].upper()
            output_file.loc[index, 'Cognome'] = name_column[0].upper()

    for index, race, time in zip(range(len(input_file['Race'])), input_file['Race'], input_file['Time']):
        for i in range(len(race)):
            output_file.loc[index, 'Gara' + str(i+1)] = race[i]
            output_file.loc[index, 'Tempo' + str(i+1)] = time[i]

    # print output_file on xlsx file
    output_file.to_excel(os.path.splitext(file_name)[0] + '.xlsx', index=False)


if __name__ == "__main__":
    print("GOandUISP v" + __version__ + " by Gregorio Berselli.")
    print("Per informazioni su come utilizzare il programma si consulti la repository GitHub: https://github.com/Grufoony/GOandUISP\n\n")
    for f in os.listdir():
        if f.endswith(".xlsx") or f.endswith(".xls"):
            convert(f)
            if not f.endswith(".xlsx"):
                os.remove(f)
            print("File " + f + " convertito con successo!")
    print("Tutti i file presenti nella cartella sono stati convertiti con successo!")
